# Language Modeling using KenLM

First off, you need to install KenLM if you haven't already. 
- Download stable release and unzip: http://kheafield.com/code/kenlm.tar.gz
- Need Boost >= 1.42.0 and bjam
    - Ubuntu: `sudo apt-get install libboost-all-dev`
    - Mac: `brew install boost; brew install bjam`
- Run within kenlm directory:
    ```bash
    mkdir -p build
    cd build
    cmake ..
    make -j 4
    ```
- `pip install https://github.com/kpu/kenlm/archive/master.zip`
- For more information on KenLM see: https://github.com/kpu/kenlm and http://kheafield.com/code/kenlm/

You've already been provided with the binaries for 2 trained language models, `nli_5gram.binary` and `sentiment.binary`. The NLI model is in the Resources/week7 folder in NYU Classes (it was too big to share on git), the Sentiment model is also available but it's also in this folder on Git.

Now, load the models!

In [1]:
import kenlm
import os

In [2]:
nli_model = kenlm.Model("nli_5gram.binary") # Or correct path to binary
sent_model =  kenlm.Model("sentiment.binary")

The KenLM model reports negative log likelihood, not perplexity. So we'll be converting the score and report net perplexity. The following function calculate the perpelxity, `get_ppl`, and find all OOV words, `get_oov`.

Pereplexity is defined as follows, $$ PPL = b^{- \frac{1}{N} \sum_{i=1}^N \log_b q(x_i)} $$
All probabilities here are in log base 10 so to convert to perplexity, we do the following $$PPL = 10^{-\log(P) / N} $$
where $P$ is the total NLL, and $N$ is the word count.

In [3]:
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        sent = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", sent)
        words = sent.strip().split()
        score = lm.score(sent, bos=False, eos=False)
        word_count = len(words)
        total_wc += word_count
        total_nll += score
    ppl = 10**-(total_nll/total_wc)
    return ppl

def get_oov(model, data):
    oov = []
    vocab = []
    for sent in data:
        sentence = sent
        words =  sentence.split()
        vocab += words
        # Find out-of-vocabulary words
        for w in words:
            if w not in model:
                    oov.append(w)
    return set(oov), set(vocab)

Function to load data,

In [4]:
def load_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            data.append(line)
    return data

 To start to get a feel for how the model performs on different sentences, try out different sentences below. (Note that this score isn't the perplexity but the negative log likelihood, you can convert it to perplexity as we did before)

In [8]:
sentence = "I am a chimpanzee ."
print(nli_model.score(sentence))
print(sent_model.score(sentence))

-12.8444375992
-20.9499568939


## Explore and experiment

Pick one to two corpora to test the language models on. Feel free to use one of following or any other appropriate corpus that appeals to you,
* Billion word dataset: http://www.statmt.org/lm-benchmark/
* Quaker historical corpus: https://www.woodbrooke.org.uk/resource-library/quaker-historical-corpus/
* All of Shakespeare: http://norvig.com/ngrams/
* IMDB: http://ai.stanford.edu/~amaas/data/sentiment/
* SNLI test set and MultiNLI dev-set (only hypothesis sentences) in data folder.

**Exercise 1:** Load the data and get the perpelxities for both models. 

**Exercise 2:** What's the difference in the size (and content) of the OOV words?

**Exercise 3:** Find the sentence with the highest difference in scores between the two models

**Exercise 4:** If you want to train a model on a larger dataset, follow the directions on the KenLM website, and see how this new model fares.